<a href="https://colab.research.google.com/github/rosselladedo/DOG_BREEDS_SEARCH_ENGINE/blob/main/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 51 kB 211 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=c33b306e593d08ab162a52f9c2ed9c1046d13b7e8bf94f3a6d14751185834cc3
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import numpy as np
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import io
import ipywidgets
import time
import anvil.server

from IPython.display import display
from ipywidgets import HBox, Image, HTML
from collections import OrderedDict

from PIL import Image
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm

In [ ]:
DOGS_BREEDS = 120
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

ALL_FEATURES_IMAGES = []

In [ ]:
#data loading
BASE_DIR = "/content/gdrive/Shareddrives/MIRCV"
IMAGES_DIR = "/content/gdrive/MyDrive/MIRCV"
NOISE_DIR = "/content/gdrive/Shareddrives/Dataset/mirflickr25k/mirflickr"

DOGS_IMAGES_DIRECTORIES_PATH = os.path.join(IMAGES_DIR, "images/Images")
ALL_IMAGES_DIR = "/content/gdrive/Shareddrives/MIRCV/All_Images"
QUERY_PATH = "/content/gdrive/Shareddrives/Dataset/Query"
FEATURES_DIR = "/content/gdrive/Shareddrives/Dataset/Features"

In [ ]:
def display_image(image_path, title='', txt=''):
  """ Displays an image with optional title and text next to it using iPython
  widgets (works in Jupyter/Colab environments).
      
  Args:
    image_path (string): path to the image to show.
    title (string): header text to show next to the image.
    txt (string): text to show under the title header.
  """
  print(str(image_path))
  image_widget = ipywidgets.Image.from_file(image_path, width=200, height=100)
  txt = txt.replace('\n', '<br>')
  text_widget = HTML('<p><b>{}</b></p><p>{}</p>'.format(title, txt))
  widget = HBox([image_widget, text_widget])
  display(widget)

In [ ]:
def take_features(path):
  features_path = np.load(path, allow_pickle=True)
  features = features_path['arr_0']
  return features

# **BRUTE FORCE**

In [ ]:
model = tf.keras.applications.densenet.DenseNet121(
    weights='imagenet', input_tensor=None, input_shape=(224,224,3), 
    pooling='avg', include_top=False
)

model.summary()

In [ ]:
#query extraction feature

def extract_features_query(img):
  query_features = []
  image_path = os.path.join(QUERY_PATH, img)
  image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
  image_pil = np.array(image_pil)
  image_pil = image_pil.astype(np.float32)
  image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
  image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
  image_feature_vector = model.predict(image_np)
  feat_name = (img, image_feature_vector)
  query_features.append(feat_name)

  np.savez("/content/gdrive/Shareddrives/Dataset/Query/features_query.npz", query_features)

In [ ]:
############################################################
#q_features = features query; g = hash families (necessaria variabile globale)
def query_execution():

  result_euclidean = []

  q_features = take_features(os.path.join(QUERY_PATH, "features_query.npz"))
  q_name = q_features[0][0]
  q_features = q_features[0][1]

  features = take_features(os.path.join(FEATURES_DIR, "features_NM_concatenate_predict.npz"))

  q_features = normalize(q_features, axis=1)
  
  for item in tqdm(features):
    feat = item[1]
    name = item[0]

    feat = normalize(feat, axis=1)

    dist = np.linalg.norm(feat - q_features, axis=1)

    res = (np.double(dist), name)

    result_euclidean.append(res)

  ranking = sorted(result_euclidean)
  
  return ranking

In [ ]:
def send_images_to_client(res):
  result_rank = []
  count = 1
  for i in res:
    name = i[1]
    #display_image(os.path.join(ALL_IMAGES_DIR, name))
    
    result_image = Image.open(open(os.path.join(ALL_IMAGES_DIR, name), 'rb'))
    #result_image = ipywidgets.Image.from_file(os.path.join(ALL_IMAGES_DIR, name), width=200, height=100)
    
    bs = io.BytesIO()
    name = "result_image"
    result_image.save(bs, format = "JPEG")
    result_rank.append(anvil.BlobMedia("image/jpeg", bs.getvalue(), name = name))
    
    if count == 10:
      break
    count = count + 1

  return result_rank

# **LSH**

In [ ]:
model_finetuned = tf.keras.models.load_model('/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model6_Flatten_512120_LR0001')

model = tf.keras.models.Model(inputs=model_finetuned.input, outputs=model_finetuned.get_layer('flatten').output)
model.summary()

In [ ]:
#query extraction feature

def extract_features_query_finetuned(img):
  query_features = []
  image_path = os.path.join(QUERY_PATH, img)
  image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
  image_pil = np.array(image_pil)
  image_pil = image_pil.astype(np.float32)
  image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
  image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
  image_feature_vector = model.predict(image_np)
  feat_name = (img, image_feature_vector)
  query_features.append(feat_name)

  np.savez("/content/gdrive/Shareddrives/Dataset/Query/features_finetuned_query.npz", query_features)

In [ ]:
#funzione che inserisce la query nei corretti buckets per ogni hash family

def insert_finetuned_query(p, g, w = 4):
  result = []
  g_dim = 1     #numero hashfamily per creare cartella relativa
  for hash_family in g:
    inserted_object = ''
    for hash_function in hash_family:
      prodotto = np.dot(p,hash_function[0])  #p*x
      somma = prodotto + hash_function[1]    #p*x + b
      divisione = somma / w             #(p*x + b)/w
      inserted_object=inserted_object+str(np.transpose(np.trunc(divisione)).astype(int))  #creao la stringa nome con tutti i buckets
    inserted_object = str(g_dim) + "/" + inserted_object
    result.append(inserted_object)

    g_dim = g_dim + 1   #incremento il numero per inserire i dati nelle altre hashfamilies
  return result

In [ ]:
############################################################
#q_features = features query; g = hash families (necessaria variabile globale)
def query_finetuned_execution(g_dim, h_dim, q_features):

  path_hash_functions = "/content/gdrive/Shareddrives/Dataset/HashFamilies/hashfunctionsG" + str(g_dim) + "H" + str(h_dim) + ".npy"
  path_bucket = "/content/gdrive/Shareddrives/Dataset/Buckets/FinetunedBuckets/bucketsG" + str(g_dim) + "H" + str(h_dim)
  
  result = []
  
  bucketsaved=np.load(path_hash_functions, allow_pickle=True)
  
  buckets_query = insert_finetuned_query(q_features[0][1], bucketsaved) #calcolo i buckets della nostra query

  for b in buckets_query:
    if os.path.exists(os.path.join(path_bucket, b)):
      file_hash=open(os.path.join(path_bucket, b),'r')
      r = file_hash.readlines()                      
      file_hash.close()
      result.append(r)

  return result

In [ ]:
#funzione per ordinare i risultati della query dal più "vicino" al più lontano

def make_finetuned_ranking(q_features, features):

  result_euclidean = []
  
  q_name = q_features[0][0]
  q_features = q_features[0][1]
  
  q_features = normalize(q_features, axis=1)

  for item in tqdm(features):
    feat = item[1]
    name = item[0]

    feat = normalize(feat, axis=1)

    dist = np.linalg.norm(feat - q_features, axis=1)

    res = (np.double(dist), name)

    result_euclidean.append(res)

  ranking = sorted(result_euclidean)
  
  return ranking

In [ ]:
#funzione per eseguire la query e stampare i risultati

def search_and_print_finetuned(g_dim,h_dim,image_name):
  print("image name query: " + str(image_name))
  #q_features=extract_features_query(image_name)
  q_features = take_features(os.path.join(QUERY_PATH, "features_finetuned_query.npz"))
  features=take_features(os.path.join(FEATURES_DIR, "features_finetuned_concatenate_predict.npz"))

  starting_time = time.time()
  query_result = query_finetuned_execution(g_dim, h_dim, q_features)
  ending_time = time.time()

  query_execution_time = ending_time - starting_time
  print("Time for the query execution: " + str(query_execution_time))

  query_result_splitted = []
  features_results=[]

  for r in query_result:
    for name in r:
      query_result_splitted.append(name.split('\n')[0])

  image_no_duplicates= list(OrderedDict.fromkeys(query_result_splitted))
  
  for f in features:
    for s in image_no_duplicates:
      if str(f[0])==s:
        features_results.append(f)

  images_to_print = make_finetuned_ranking(q_features, features_results)

  return images_to_print

In [ ]:
def send_images_to_client_finetuned(res):
  result_rank = []
  count = 1
  for i in res:
    name = i[1]
    #display_image(os.path.join(ALL_IMAGES_DIR, name))
    
    result_image = Image.open(open(os.path.join(ALL_IMAGES_DIR, name), 'rb'))
    #result_image = ipywidgets.Image.from_file(os.path.join(ALL_IMAGES_DIR, name), width=200, height=100)
    
    bs = io.BytesIO()
    name = "result_image"
    result_image.save(bs, format = "JPEG")
    result_rank.append(anvil.BlobMedia("image/jpeg", bs.getvalue(), name = name))
    
    if count == 10:
      break
    count = count + 1

  return result_rank

# **SERVER LAUNCH**

In [ ]:
#anvil.server.connect("REZ4P23PNYAA3PDVGAGWE74S-K44TCW6TYVNHOTGF") #gaetano
#anvil.server.connect("UETT2ZYAYVLZHOZE5SUFOVGG-GKQWYSJZOMLZMMVZ") #ross
anvil.server.connect("APYM2HWOGSI6BTH4XYVWSAV2-XSCRAS7EJA7HUPIO") #Giacomo

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
@anvil.server.callable
def process_image(image, image_name):

  image_object = Image.open(io.BytesIO(image.get_bytes()))

  image_object.save(os.path.join(QUERY_PATH, image_name))
  
  print("Extracting query features...")
  extract_features_query(image_name)
  #extract_features_query_finetuned(image_name)
  print("Query features extracted!")

  query_result = query_execution()
  #query_result = search_and_print_finetuned(8, 2, image_name)
  print("Query executed!")

  result_to_be_sent = send_images_to_client(query_result)
  #result_to_be_sent = send_images_to_client_finetuned(query_result)
  print("Results ready!")

  os.remove(os.path.join(QUERY_PATH, image_name))
  os.remove(os.path.join(QUERY_PATH, "features_query.npz"))
  print("Query directory cleaned!")

  return result_to_be_sent

In [ ]:
if __name__ == "__main__":
  try:
    anvil.server.wait_forever()
  except KeyboardInterrupt:
    print("Bona cisi e se un cisi bona")


Extracting query features...


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Query features extracted!


  0%|          | 0/44580 [00:00<?, ?it/s]

Query executed!
Results ready!
Query directory cleaned!
Extracting query features...


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Query features extracted!


  0%|          | 0/44580 [00:00<?, ?it/s]

Query executed!
Results ready!
Query directory cleaned!
Bona cisi e se un cisi bona
